# Importación de librerías

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [116]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import spacy
nlp = spacy.load('es_core_news_sm')


# Cargamos el dataset

In [117]:
#df = pd.read_csv('trackings_fechas_v07-09-24.csv', sep=';')
df = pd.read_csv('ml 1.csv', sep=';')

In [118]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
columnas = df.columns
print(columnas)

# Preparación de datos
## Unificación de comentarios

In [ ]:
df_grouped = df.groupby('IdTracking').apply(
    lambda x: pd.Series({
        'Des': x['Des'].iloc[0],  # Cargar 'Des' solo una vez (el primer valor del grupo)
        'Des_Combined': ' '.join(x['Des.1'].astype(str)),
        'IdTipoTracking': x['IdTipoTracking'].iloc[0],
        'IdPrioridad': x['IdPrioridad'].iloc[0],
        'IdTipoTarea': x['IdTipoTarea'].iloc[0],
        'IdComponente': x['IdComponente'].iloc[0],
        'IdPropuesta': x['IdPropuesta'].iloc[0],
        'IdCuenta': x['IdCuenta'].iloc[0],
        'FecAlt': x['FecAlt'].iloc[0],
        'IdUsuarioAlt': x['IdUsuarioAlt'].iloc[0],
        'FecAsi': x['FecAsi'].iloc[0],
        'IdUsuarioAsi': x['IdUsuarioAsi'].iloc[0],
        'IdEstado': x['IdEstado'].iloc[0],
        'IdTipoEstado': x['IdTipoEstado'].iloc[0],
        'HorEst': x['HorEst'].iloc[0],
        'MinEst': x['MinEst'].iloc[0],
        'Inter': x['Inter'].iloc[0],
        'Exter': x['Exter'].iloc[0],
        'IdHito': x['IdHito'].iloc[0],
        'IdVersion': x['IdVersion'].iloc[0],
        'FecVto': x['FecVto'].iloc[0],
        'HorDes': x['HorDes'].iloc[0],
        'IdProducto': x['IdProducto'].iloc[0],
        'IdCategoria': x['IdCategoria'].iloc[0],
        'AplMan': x['AplMan'].iloc[0],
        'Hordesdes': x['Hordesdes'].iloc[0],
        'Hordestes': x['Hordestes'].iloc[0],
        'Horesttes': x['Horesttes'].iloc[0],
        'Horestdes': x['Horestdes'].iloc[0],
        'HorCli': x['HorCli'].iloc[0],
        'ConCan': x['ConCan'].iloc[0],
        'Obs': x['Obs'].iloc[0],
        'IdTareaProyecto': x['IdTareaProyecto'].iloc[0],
        'IdSubtipoProducto': x['IdSubtipoProducto'].iloc[0],
        'IdTrackingRelacionado': x['IdTrackingRelacionado'].iloc[0],
        'IdComplejidad': x['IdComplejidad'].iloc[0],
        'FecUltMod': x['FecUltMod'].iloc[0],
        'IdServidor': x['IdServidor'].iloc[0],
        'Facturable': x['Facturable'].iloc[0]
    })
).reset_index()

df_grouped.head()


In [ ]:
columnas = df_grouped.columns
print(columnas)

In [124]:
df_grouped['Des_total']= df_grouped['Des'] + ' ' + df_grouped['Des_Combined']

In [ ]:
df_grouped.head()

In [ ]:
# Eliminar las columnas 'Des' y 'Des.1' del DataFrame
df_grouped = df_grouped.drop(columns=['Des', 'Des_Combined'])

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_grouped.head()

In [ ]:
# Reorganizar las columnas para mover 'Des_total' al lado de 'IdTracking'
cols = ['IdTracking', 'Des_total'] + [col for col in df_grouped.columns if col not in ['IdTracking', 'Des_total']]
df_grouped = df_grouped[cols]

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_grouped.head()

In [ ]:
df_grouped.head()

In [ ]:
df_filtered = df_grouped[df_grouped['IdTracking'] == 38593]
pd.set_option('display.max_colwidth', None)

#Mostrar las primeras filas del DataFrame filtrado
df_filtered.head()

Se realiza la imputacion de valores a la variable facturable

In [ ]:
# Contar la cantidad de registros en el DataFrame 'df_grouped'
num_records = df_grouped.shape[0]

# Mostrar el resultado
print(f"Número de registros en df_grouped: {num_records}")

In [ ]:
#visualizamos la columna facturable
df_grouped['Facturable'].value_counts()
#reemplamos los #n/a por 0


In [ ]:
# Contar la cantidad de valores únicos en la columna 'TrackingID'
num_distinct_tracking_ids = df['IdTracking'].nunique()

# Mostrar el resultado
print(f"Número de TrackingID distintos: {num_distinct_tracking_ids}")


In [ ]:
df_grouped['Facturable'] = df_grouped['Facturable'].replace('#N/D', 0)
df_grouped['Facturable'].value_counts()

### EDA

In [ ]:
# Tipos de datos
df_grouped.dtypes

In [ ]:
# Verificamos el porcentaje de las clases de la columna 'Facturable'
df_grouped['Facturable'].value_counts(normalize=True)


In [ ]:
#Graficamos la distribución de la columna 'Facturable', en color naranja los no facturables y en azul los facturables
sns.countplot(x='Facturable', data=df_grouped, palette='coolwarm')
plt.xticks(ticks=[0,1],labels=['NO', 'SI'])
plt.title('Distribución de la variable Facturable')
plt.show()


In [ ]:
# Diccionario de mapeo para 'IdTipoTracking'
mapeo_idtipo_tracking = {
    1: 'Error',
    5: 'Consulta',
    4: 'Requerimiento',
    8: 'Entregable',
    2: 'Nueva tarea',
    7: 'Implementacion',
    9: 'Caso Prueba',
    6: 'Seguimiento/Gestión'
}

# Diccionario de mapeo para 'IdPrioridad'
mapeo_idprioridad = {
    5: 'Critico',
    3: 'Alta',
    1: 'Baja',
    2: 'Media',
    4: 'Urgente'
}

# Mapeo para la columna 'Facturable'
mapeo_facturable = {0: 'NO', 1: 'SI'}

# Graficamos los countplots
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico para IdTipoTracking
sns.countplot(x='IdTipoTracking', hue='Facturable', data=df_grouped, ax=axs[0], palette='coolwarm')

# Gráfico para IdPrioridad
sns.countplot(x='IdPrioridad', hue='Facturable', data=df_grouped, ax=axs[1], palette='coolwarm')

# Reemplazamos los valores de 'IdTipoTracking' por sus descripciones en el gráfico
axs[0].set_xticks(range(len(mapeo_idtipo_tracking)))  # Aseguramos que los ticks se alineen
axs[0].set_xticklabels([mapeo_idtipo_tracking[i] for i in sorted(mapeo_idtipo_tracking.keys())], rotation=90, ha='center')

# Reemplazamos los valores de 'IdPrioridad' por sus descripciones en el gráfico
axs[1].set_xticks(range(len(mapeo_idprioridad)))  # Aseguramos que los ticks se alineen
axs[1].set_xticklabels([mapeo_idprioridad[i] for i in sorted(mapeo_idprioridad.keys())], rotation=90, ha='center')

# Reemplazamos los valores de 'Facturable' por 'SI' y 'NO'
handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles=handles, labels=[mapeo_facturable[int(label)] for label in labels])

handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles=handles, labels=[mapeo_facturable[int(label)] for label in labels])

plt.tight_layout()
plt.show()

# Agregamos líneas de densidad para 'IdTipoTracking' y 'IdPrioridad' según 'Facturable'
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de densidad para 'IdTipoTracking'
sns.kdeplot(data=df_grouped, x='IdTipoTracking', hue='Facturable', ax=axs[0], palette='coolwarm')

# Gráfico de densidad para 'IdPrioridad'
sns.kdeplot(data=df_grouped, x='IdPrioridad', hue='Facturable', ax=axs[1], palette='coolwarm')

# Reemplazamos los valores de 'IdTipoTracking' por sus descripciones en el gráfico
axs[0].set_xticks(range(len(mapeo_idtipo_tracking)))  # Aseguramos que los ticks se alineen
axs[0].set_xticklabels([mapeo_idtipo_tracking[i] for i in sorted(mapeo_idtipo_tracking.keys())], rotation=90, ha='center')

# Reemplazamos los valores de 'IdPrioridad' por sus descripciones en el gráfico
axs[1].set_xticks(range(len(mapeo_idprioridad)))  # Aseguramos que los ticks se alineen
axs[1].set_xticklabels([mapeo_idprioridad[i] for i in sorted(mapeo_idprioridad.keys())], rotation=90, ha='center')

# Reemplazamos los valores de 'Facturable' por 'SI' y 'NO'
handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend(handles=handles, labels=[mapeo_facturable[int(label)] for label in labels])

handles, labels = axs[1].get_legend_handles_labels()
axs[1].legend(handles=handles, labels=[mapeo_facturable[int(label)] for label in labels])

plt.tight_layout()
plt.show()


In [ ]:
# Diccionario de mapeo para 'IdTipoTracking'
mapeo_idtipo_tracking = {
    1: 'Error',
    5: 'Consulta',
    4: 'Requerimiento',
    8: 'Entregable',
    2: 'Nueva tarea',
    7: 'Implementacion',
    9: 'Caso Prueba',
    6: 'Seguimiento/Gestión'
}

# Diccionario de mapeo para 'IdPrioridad'
mapeo_idprioridad = {
    5: 'Critico',
    3: 'Alta',
    1: 'Baja',
    2: 'Media',
    4: 'Urgente'
}

# Mapeo para la columna 'Facturable'
mapeo_facturable = {0: 'NO', 1: 'SI'}

# Ahora graficamos en barras apiladas la proporción de facturables según IdTipoTracking e IdPrioridad
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Crosstab para 'IdTipoTracking' y 'Facturable'
crosstab_idtipotracking = pd.crosstab(df_grouped['IdTipoTracking'], df_grouped['Facturable']).rename(index=mapeo_idtipo_tracking)
crosstab_idtipotracking.columns = [mapeo_facturable[col] for col in crosstab_idtipotracking.columns]
crosstab_idtipotracking.plot(kind='bar', stacked=True, ax=axs[0], color=['orange', 'blue'])

# Crosstab para 'IdPrioridad' y 'Facturable'
crosstab_idprioridad = pd.crosstab(df_grouped['IdPrioridad'], df_grouped['Facturable']).rename(index=mapeo_idprioridad)
crosstab_idprioridad.columns = [mapeo_facturable[col] for col in crosstab_idprioridad.columns]
crosstab_idprioridad.plot(kind='bar', stacked=True, ax=axs[1], color=['orange', 'blue'])

# Etiquetas de los ejes y el título de las gráficas
axs[0].set_xlabel('IdTipoTracking')
axs[0].set_ylabel('Cantidad')
axs[0].set_title('Proporción de Facturables por IdTipoTracking')

axs[1].set_xlabel('IdPrioridad')
axs[1].set_ylabel('Cantidad')
axs[1].set_title('Proporción de Facturables por IdPrioridad')

plt.tight_layout()
plt.show()


In [ ]:
#Ahora repetimos el proceso pero con un grafico de torta, para la clase Facturable=1, IdTipoTracking e IdPrioridad
# Diccionario de mapeo para 'IdTipoTracking'
mapeo_idtipo_tracking = {
    1: 'Error',
    5: 'Consulta',
    4: 'Requerimiento',
    8: 'Entregable',
    2: 'Nueva tarea',
    7: 'Implementacion',
    9: 'Caso Prueba',
    6: 'Seguimiento/Gestión'
}

# Diccionario de mapeo para 'IdPrioridad'
mapeo_idprioridad = {
    5: 'Critico',
    3: 'Alta',
    1: 'Baja',
    2: 'Media',
    4: 'Urgente'
}

# Filtramos el DataFrame para los facturables (Facturable == 1)
df_facturable = df_grouped[df_grouped['Facturable'] == 1]

# Creamos los gráficos de pastel
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de pastel para 'IdTipoTracking' con mapeo
df_facturable['IdTipoTracking'].replace(mapeo_idtipo_tracking).value_counts().plot(kind='pie', autopct='%1.1f%%', ax=axs[0])
axs[0].set_ylabel('')  # Removemos la etiqueta del eje y para mejorar visualización
axs[0].set_title('Distribución de IdTipoTracking (Facturables)')

# Gráfico de pastel para 'IdPrioridad' con mapeo
df_facturable['IdPrioridad'].replace(mapeo_idprioridad).value_counts().plot(kind='pie', autopct='%1.1f%%', ax=axs[1])
axs[1].set_ylabel('')  # Removemos la etiqueta del eje y para mejorar visualización
axs[1].set_title('Distribución de IdPrioridad (Facturables)')

plt.tight_layout()
plt.show()






In [ ]:
#Ahora listado de la clase Facturables=1 según IdTipoTracking e IdPrioridad, de forma decreciente 

df_grouped[df_grouped['Facturable'] == 1]['IdTipoTracking'].value_counts().sort_values(ascending=False)


In [ ]:
df_grouped[df_grouped['Facturable'] == 1]['IdPrioridad'].value_counts().sort_values(ascending=False)

### Feature Engineering

Creación de la Variable "TiempoTranscurrido"

In [ ]:
# Creamos la variable temporal que proviene del tiempo transcurrido entre la fecha de alta y la fecha de vencimiento
df_grouped['FecAlt'] = pd.to_datetime(df_grouped['FecAlt'], format='%d/%m/%Y %I:%M:%S %p')
df_grouped['FecUltMod'] = pd.to_datetime(df_grouped['FecUltMod'], format='%d/%m/%Y %I:%M:%S %p')

df_grouped['TiempoTranscurrido'] = (df_grouped['FecUltMod'] - df_grouped['FecAlt']).dt.days
 
# Mostramos las primeras filas del DataFrame para verificar el resultado
df_grouped.head()


In [ ]:
#Verificamos los parametros de la variable 'TiempoTranscurrido'
df_grouped['TiempoTranscurrido'].describe()


In [ ]:
#Vemos su distribución
sns.histplot(df_grouped['TiempoTranscurrido'], bins=50, kde=True)
plt.title('Distribución de la variable TiempoTranscurrido')
plt.show()

In [ ]:
#Identificamos los registros negativos de 'TiempoTranscurrido'
df_grouped[df_grouped['TiempoTranscurrido'] < 0]

#Visualizamos los registros negativos de 'TiempoTranscurrido'
df_grouped[df_grouped['TiempoTranscurrido'] < 0].head()

#Asignamos el valor 0 a los registros negativos de 'TiempoTranscurrido', ya que corresponden a un mismo día
df_grouped.loc[df_grouped['TiempoTranscurrido'] < 0, 'TiempoTranscurrido'] = 0

#Verificamos que no existan registros negativos en 'TiempoTranscurrido'
df_grouped['TiempoTranscurrido'].min()

df_grouped['TiempoTranscurrido'].describe()


In [ ]:
#Graficamos la distribución de la variable 'TiempoTranscurrido', según la variable 'Facturable 
sns.histplot(data=df_grouped, x='TiempoTranscurrido', hue='Facturable', bins=50, kde=True)
plt.title('Distribución de la variable TiempoTranscurrido según Facturable')
plt.show()

In [ ]:
#ahora indicamos la cantidad de registros con 'TiempoTranscurrido'ordenados de forma descendente
df_grouped['TiempoTranscurrido'].value_counts().sort_values(ascending=False)

In [ ]:
#Ahora armamos una tabla para el total de registros por cada valor de 'TiempoTranscurrido', según la clase 'Facturable'
pd.crosstab(df_grouped['TiempoTranscurrido'], df_grouped['Facturable'])


In [ ]:
#Indicamos la fracción de facturable 1 y 0 para cada valor de 'TiempoTranscurrido'
pd.crosstab(df_grouped['TiempoTranscurrido'], df_grouped['Facturable'], normalize='index')
